In [10]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline

def Evaluate(pre_disp_path, gt_depth_path,name):
    min_depth = 1e-3
    max_depth = 1
    predicted_disp_path = pre_disp_path
    gt_path = gt_depth_path
    
    rms     = np.zeros(num_samples, np.float32)
    log_rms = np.zeros(num_samples, np.float32)
    abs_rel = np.zeros(num_samples, np.float32)
    sq_rel  = np.zeros(num_samples, np.float32)
    d1_all  = np.zeros(num_samples, np.float32)
    a1      = np.zeros(num_samples, np.float32)
    a2      = np.zeros(num_samples, np.float32)
    a3      = np.zeros(num_samples, np.float32)


    def compute_errors(gt, pred):
        thresh = np.maximum((gt / pred), (pred / gt))
        a1 = (thresh < 1.25   ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())
        rmse_log = (np.log(gt) - np.log(pred)) ** 2
        rmse_log = np.sqrt(rmse_log.mean())
        abs_rel = np.mean(np.abs(gt - pred))
        sq_rel = np.mean(((gt - pred)**2))

        return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3
    

    def load_gt_depth_kitti(path, num_samples):
        gt_depths = []
        for i in range (num_samples):
            depth = cv2.imread(path + str(i).zfill(5) + ".png", -1)
            depth = depth.astype(np.float32) / 256
            gt_depths.append(depth)
        return gt_depths

    def predict_disps_resize(gt_depth, pred_disparities):
        pred_disparities_resized = []
        for i in range(len(gt_depth)):
            height, width = gt_depth[i].shape
            pred_disp = pred_disparities[i]
            pred_disp = width * cv2.resize(pred_disp, (width, height), interpolation=cv2.INTER_LINEAR)
            pred_disp = pred_disp.astype(np.float32) / 256
            pred_disparities_resized.append(pred_disp) 
            np.set_printoptions(threshold=np.inf)

        return pred_disparities_resized

    gt_depths = load_gt_depth_kitti(gt_path, num_samples)

    pred_disparities = np.load(predicted_disp_path)
    pred_disparities_resized = predict_disps_resize(gt_depths, pred_disparities)
    for i in range(num_samples):
        gt_depth = gt_depths[i]
        #print("gt_depth...............................",gt_depth.max())
        pred_disparities_resized_a = pred_disparities_resized[i]
        #print("pred_disparities_resized_a.............",pred_disparities_resized_a.max())
        gt_depth[gt_depth < min_depth] = min_depth
        pred_disparities_resized_a[pred_disparities_resized_a < min_depth] = min_depth
        pred_disparities_resized_a[pred_disparities_resized_a > max_depth] = max_depth
        
        abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i] = compute_errors(gt_depth,pred_disparities_resized_a)
   
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format
          ('abs_rel', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
    print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format
          (abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))
    
    outfile="/home/nung/Desktop/evaluate/evaluate.txt"
    file=open(outfile, "a")
    file.write('200frame'+' '+name+'\n'+str(abs_rel.mean())+' '+str(sq_rel.mean())+' '+str(rms.mean())+' '+str(log_rms.mean())+' '+
               str(a1.mean())+' '+str(a2.mean())+' '+str(a3.mean())+'\n')
    
def Go_driving0_5():
    name='test_whole_1'
    print(name+'/'+str(num_samples)+' testing')
    predicted_disp_path = '/home/nung/gradient/test_whole_1/disparities_pp.npy'
    Evaluate(predicted_disp_path,gt_path, name)

    
num_samples = 88
gt_path = '/home/nung/data/driving/data/test/'
Go_driving0_5()

test_whole_1/88 testing
   abs_rel,     sq_rel,        rms,    log_rms,         a1,         a2,         a3
    0.0668,     0.0205,      0.117,      1.075,      0.675,      0.791,      0.834
